Import libraries

In [57]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import string

Load the data

In [71]:
datas = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test.shape

(2386, 1)

مرتب سازی دیتا

In [59]:
datas.drop_duplicates (inplace = True)
test.drop_duplicates (inplace = True)
datas.columns
datas.isnull().sum()

#Need to download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/afla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
def process_text(text):
    
    #1 Remove Punctuationa
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    #2 Remove Stop Words
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    #3 Return a list of clean words
    return clean_words

In [72]:
datas['Text'].head().apply(process_text)
test['Text'].head().apply(process_text)
test.shape

(2386, 1)

In [62]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(decode_error='replace', encoding='utf-8')
v2 = CountVectorizer(decode_error='replace', encoding='utf-8')

messages_bow = v.fit_transform(datas['Text'].values.astype('U'))
messages_bow2 = v2.fit_transform(test['Text'].values.astype('U'))
#print(messages_bow2.shape)
#print(messages_bow)
#from sklearn.feature_extraction.text import CountVectorizer
#messages_bow = CountVectorizer(analyzer=process_text).fit_transform(df['Text'])
messages_bow.shape

(8307, 98960)

TRAiNING...

In [63]:
#Split data into 80% training & 20% testing data sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(datas['Text'], datas['Class'], test_size = 0.20, random_state = 0)

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(decode_error='replace', encoding='utf-8')
X_train1 = vectorizer.fit_transform(X_train)
X_test1 = vectorizer.transform(X_test.values.astype('U'))

test1 = vectorizer.transform(test['Text'])

test1.shape

(2386, 80941)

In [65]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train1, y_train)

pred = classifier.predict(X_test1)
print(y_train.values,classifier.predict(X_train1))

[1 0 1 ... 1 1 1] [1 0 1 ... 1 1 1]


In [66]:
#Evaluate the model on the training data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_train1)
print(classification_report(y_train ,pred))
print('***','Confusion Matrix: \n',confusion_matrix(y_train,pred))
print()
print('Accuracy: ', accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      3744
           1       0.97      1.00      0.98      2901

    accuracy                           0.98      6645
   macro avg       0.98      0.99      0.98      6645
weighted avg       0.99      0.98      0.98      6645

*** Confusion Matrix: 
 [[3651   93]
 [   9 2892]]

Accuracy:  0.9846501128668171


In [67]:
#Print the predictions
print('Predicted value: ',classifier.predict(X_test1))
#Print Actual Label
print('Actual value: ',y_test.values)

Predicted value:  [1 1 0 ... 0 1 0]
Actual value:  [1 0 0 ... 0 1 0]


In [68]:
#Evaluate the model on the test data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_test1)
print(pred)
print(classification_report(y_test ,pred ))
print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred))
print('-'*65)
m = classifier.predict(test1)
n = classifier.predict_proba(test1)
#print(classifier.predict_proba(test1))


[1 1 0 ... 0 1 0]
              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1000
           1       0.90      0.99      0.94       662

    accuracy                           0.95      1662
   macro avg       0.94      0.96      0.95      1662
weighted avg       0.95      0.95      0.95      1662

Confusion Matrix: 
 [[927  73]
 [  9 653]]

Accuracy:  0.950661853188929
-----------------------------------------------------------------


In [76]:
n = classifier.predict_proba(test1)
#print(n)
#print('per: ',n[:,1])
import csv
#outfile = open('/home/afla/Documents/python/poroje1- khordad/مزاحمین داده‌ای/submission.csv','w')
#out = csv.writer(outfile)
#out.writerows(map(lambda x: [x], n[:,1]))
#outfile.close()
h = np.array(['Class'])
print()
P = n[:,1]
s = np.append(h, P)
print(s,'***')
import pandas as pd
df = pd.DataFrame(s)
df.to_csv('submission.csv')
print(n.shape,s.shape)


['Class' '0.9999924567261479' '0.9999987839287461' ...
 '0.9999921298486684' '0.04416820256647981' '0.05925164158110207'] ***
(2386, 2) (2387,)
